In [ ]:
def __G(self, x):
    gamma = 0.5 - self.H
    F = scipy.special.hyp2f1(1.0,gamma,2.0-gamma,1.0/x)
    return (1.0-2.0*gamma)/(1.0-gamma)*x**(-gamma)*F

def __check_symmetric(self, a):
    return np.allclose(a, a.T)

def WZ(self):
    idx = self.t
    v, u = np.zeros(pow(self.s,2)), np.zeros(pow(self.s,2))

    k = 0
    for i in product(idx, idx):
        v[k], u[k] = i
        k = k + 1

    D = np.sqrt(2.0*self.H)/(self.H+0.5)
    res = self.rho * D * (v**(self.H+0.5)-(v-np.minimum(u,v))**(self.H+0.5))
    return np.reshape(res, (self.s,self.s))

def WW(self):
    cov = np.zeros((self.s,self.s))

    i = 0
    for v in self.t:
        j = 0
        for u in self.t:
            if i == j:
                cov[i,j] = u**(2.0*self.H)
            if i > j:
                x = v/u
                cov[i,j] = u**(2.0*self.H)*self.__G(x)
            if i < j:
                cov[i,j] = 0
            j += 1
        i += 1

    return cov + cov.T - np.diag(cov.diagonal())

def ZZ(self):
    cov = np.zeros((self.s,self.s))

    i = 0
    for v in self.t:
        j = 0
        for u in self.t:
            cov[i,j] = np.min([v,u])
            j += 1
        i += 1

    return cov

def joint_cov(self):
    res = np.block([[self.WW(), self.WZ()],[self.WZ().T, self.ZZ()]])
    if self.__check_symmetric(res):
        return res
    else:
        print('Covariance matrix is not symmetric...')

def paths(self):
    normal = np.random.rand(self.s*2,self.npaths)
    cov = self.joint_cov()
    sigma = scipy.linalg.cholesky(cov,lower=True)
    control = np.all(np.matmul(sigma,sigma.T) - cov < 1e-06)
    if control:
        print("Good news, everyone! I think I perfected a scheme that will simulate all paths!")
        return np.matmul(sigma, normal)
    else: 
        print("Bad news, everyone! I don't think the simulation is going to make it...")